In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv
/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

train = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktrain.csv')
test = pd.read_csv('/kaggle/input/summer-analytics-mid-hackathon/hacktest.csv')

In [21]:
# Fill missing values by forward and backward filling
train_filled = train.iloc[:, 2:].apply(pd.to_numeric, errors='coerce').ffill(axis=1).bfill(axis=1)
test_filled = test.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').ffill(axis=1).bfill(axis=1)

def create_features(df):
    features = pd.DataFrame()
    features['mean_ndvi'] = df.mean(axis=1)
    features['median_ndvi'] = df.median(axis=1)
    features['std_ndvi'] = df.std(axis=1)
    features['max_ndvi'] = df.max(axis=1)
    features['min_ndvi'] = df.min(axis=1)
    features['range_ndvi'] = features['max_ndvi'] - features['min_ndvi']
    features['slope'] = df.apply(lambda x: np.polyfit(range(len(x)), x, 1)[0], axis=1)
    
    n = len(df.columns)
    features['summer_avg'] = df.iloc[:, n//4 : 3*n//4].mean(axis=1)
    features['winter_avg'] = (df.iloc[:, :n//4].mean(axis=1) + df.iloc[:, 3*n//4:].mean(axis=1)) / 2
    
    return features

train_features = create_features(train_filled)
test_features = create_features(test_filled)

X = train_features
y = train['class']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
model.fit(X_train_scaled, y_train)

val_preds = model.predict(X_val_scaled)
print("Validation Accuracy:", accuracy_score(y_val, val_preds))

test_scaled = scaler.transform(test_features)
test_preds = model.predict(test_scaled)


Validation Accuracy: 0.85875


In [22]:
# Saving to CSV:
submission = pd.DataFrame({'ID': test['ID'], 'class': test_preds})
submission.to_csv('submission.csv', index=False)